This quickstart notebook allows to test and mess around with the MLPF GNN model in a standalone way. For actual training, we don't use a notebook, please refer to `README.md`.


```bash
git clone https://github.com/jpata/particleflow/
```

Run the notebook from `notebooks/delphes-tf-mlpf-quickstart.ipynb`.

In [ ]:
import bz2, pickle
import numpy as np
import tensorflow as tf
import sklearn
import sklearn.metrics
import matplotlib.pyplot as plt
import yaml

tf.config.run_functions_eagerly(False)

In [ ]:
import sys

sys.path += ["../mlpf", "../hep_tfds"]

In [ ]:
import tfmodel
from tfmodel.model_setup import make_gnn_dense

In [ ]:
!wget --no-check-certificate -nc https://zenodo.org/record/4452283/files/tev14_pythia8_ttbar_0_0.pkl.bz2

In [ ]:
data = pickle.load(bz2.BZ2File("tev14_pythia8_ttbar_0_0.pkl.bz2", "r"))

In [ ]:
# 100 events in one file
len(data["X"]), len(data["ygen"])

In [ ]:
# Pad the number of elements to a size that's divisible by the bin size
Xs = []
ys = []

max_size = 50 * 128
for i in range(len(data["X"])):
    X = data["X"][i][:max_size, :]
    y = data["ygen"][i][:max_size, :]
    Xpad = np.pad(X, [(0, max_size - X.shape[0]), (0, 0)])
    ypad = np.pad(y, [(0, max_size - y.shape[0]), (0, 0)])
    Xpad = Xpad.astype(np.float32)
    ypad = ypad.astype(np.float32)
    Xs.append(Xpad)
    ys.append(ypad)

X = np.stack(Xs)[:10]
y = np.stack(ys)[:10]

In [ ]:
# Get the first event
input_classes = np.unique(X[:, :, 0].flatten())
output_classes = np.unique(y[:, :, 0].flatten())
num_output_classes = len(output_classes)

In [ ]:
input_classes

In [ ]:
output_classes

In [ ]:
def transform_target(y):
    return {
        "cls": tf.one_hot(tf.cast(y[:, :, 0], tf.int32), num_output_classes),
        "charge": y[:, :, 1:2],
        "pt": y[:, :, 2:3],
        "eta": y[:, :, 3:4],
        "sin_phi": y[:, :, 4:5],
        "cos_phi": y[:, :, 5:6],
        "energy": y[:, :, 6:7],
    }


yt = transform_target(y)

In [ ]:
msk_true_particle = y[:, :, 0] != 0

In [ ]:
np.unique(y[msk_true_particle][:, 0], return_counts=True)

In [ ]:
plt.hist(yt["pt"][msk_true_particle].flatten(), bins=100)
plt.xlabel("pt")
plt.yscale("log")

In [ ]:
plt.hist(yt["eta"][msk_true_particle].flatten(), bins=100)
plt.xlabel("eta")

In [ ]:
plt.hist(yt["sin_phi"][msk_true_particle].flatten(), bins=100)
plt.xlabel("sin phi")

In [ ]:
plt.hist(yt["cos_phi"][msk_true_particle].flatten(), bins=100)
plt.xlabel("cos phi")

In [ ]:
plt.hist(yt["energy"][msk_true_particle].flatten(), bins=100)
plt.xlabel("energy")
plt.yscale("log")

In [ ]:
with open("../parameters/delphes.yaml", "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)
    config["setup"]["multi_output"] = True

In [ ]:
model = PFNetDense(
    num_input_classes=len(input_classes),
    num_output_classes=len(output_classes),
    activation="elu",
    hidden_dim=128,
    bin_size=128,
    input_encoding="default",
    multi_output=True,
    max_bin_size=100,
    combined_graph_layer={
        "bin_size": 640,
        "max_num_bins": 100,
        "distance_dim": 128,
        "layernorm": False,
        "num_node_messages": 1,
        "dropout": 0.0,
        "dist_activation": "linear",
        "ffn_dist_num_layers": 1,
        "ffn_dist_hidden_dim": 128,
        "kernel": {"type": "NodePairGaussianKernel", "dist_mult": 0.1, "clip_value_low": 0.0, "dist_norm": "l2"},
        "node_message": {"type": "GHConvDense", "output_dim": 256, "activation": "elu", "normalize_degrees": True},
        "activation": "elu",
    },
)

# #temporal weight mode means each input element in the event can get a separate weight
model.compile(
    loss={
        "cls": tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        "charge": tf.keras.losses.MeanSquaredError(),
        "pt": tf.keras.losses.MeanSquaredError(),
        "energy": tf.keras.losses.MeanSquaredError(),
        "eta": tf.keras.losses.MeanSquaredError(),
        "sin_phi": tf.keras.losses.MeanSquaredError(),
        "cos_phi": tf.keras.losses.MeanSquaredError(),
    },
    optimizer="adam",
    sample_weight_mode="temporal",
)

In [ ]:
model.fit(X, yt, epochs=2, batch_size=1)

In [ ]:
ypred = model.predict(X, batch_size=5)

In [ ]:
# index of the class prediction output values
pred_id_offset = len(output_classes)
ypred_ids_raw = ypred["cls"]

In [ ]:
sklearn.metrics.confusion_matrix(
    np.argmax(ypred_ids_raw, axis=-1).flatten(), np.argmax(yt["cls"], axis=-1).flatten(), labels=output_classes
)

In [ ]:
msk_particles = X[:, :, 0] != 0
plt.scatter(ypred["eta"][msk_particles].flatten(), yt["eta"][msk_particles].flatten(), marker=".")